Initial Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
assist_df = pd.read_csv('/content/drive/MyDrive/Education_Data/ASSIST 2012-2013/2012-2013-data-with-predictions-4-final.csv', encoding = "ISO-8859-1")
cols = assist_df.columns
print(cols)
assist_df

Index(['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id',
       'assistment_id', 'start_time', 'end_time', 'problem_type', 'original',
       'correct', 'bottom_hint', 'hint_count', 'actions', 'attempt_count',
       'ms_first_response', 'tutor_mode', 'sequence_id', 'student_class_id',
       'position', 'type', 'base_sequence_id', 'skill_id', 'teacher_id',
       'school_id', 'overlap_time', 'template_id', 'answer_id', 'answer_text',
       'first_action', 'problemlogid', 'Average_confidence(FRUSTRATED)',
       'Average_confidence(CONFUSED)', 'Average_confidence(CONCENTRATING)',
       'Average_confidence(BORED)'],
      dtype='object')


,problem_log_id,skill,problem_id,user_id,assignment_id,assistment_id,start_time,end_time,problem_type,original,...,overlap_time,template_id,answer_id,answer_text,first_action,problemlogid,Average_confidence(FRUSTRATED),Average_confidence(CONFUSED),Average_confidence(CONCENTRATING),Average_confidence(BORED)
0,137792159,NaN,557460,61394,565736,341511,2012-09-28 15:11:27,2012-09-28 15:11:36.856,choose_1,1,...,9852,341511,NaN,she,0,137792159,0.361323,0.0,0.336529,0.000000
1,138083797,Rounding,365981,61394,573819,204043,2012-10-09 11:01:52,2012-10-09 11:02:13.182,algebra,1,...,21175,204043,NaN,74.29,0,138083797,0.361323,0.0,0.766925,0.000000
2,142332619,Multiplication and Division Integers,426415,61394,734130,247525,2013-03-07 10:53:20,2013-03-07 10:53:28.661,algebra,1,...,8645,247525,NaN,00,0,142332619,0.361323,0.0,0.766925,0.442968
3,145939397,Proportion,86686,61394,821352,48081,2013-08-20 19:54:56,2013-08-20 19:55:21.753,algebra,1,...,25728,46362,NaN,3.8,0,145939397,0.775000,0.0,0.766925,0.912281
4,137111284,NaN,399669,76592,557216,227869,2012-09-10 17:20:10,2012-09-10 17:24:56.579,choose_1,1,...,286578,227869,NaN,C (wr - 1)(wr + 1),0,137111284,0.361323,0.0,0.766925,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6123265,146114936,NaN,330756,227984,832474,180547,2013-08-30 10:10:55,2013-08-30 10:11:08.935,choose_1,1,...,13919,180547,NaN,yes,0,146114936,0.361323,0.0,0.766925,0.000000
6123266,146131268,NaN,765964,228009,831417,491711,2013-08-31 10:23:24,2013-08-31 10:23:27.757,choose_1,1,...,3748,491711,NaN,Biotic factors in a forest,0,146131268,0.361323,0.0,0.766925,0.000000
6123267,146118011,Addition and Subtraction Integers,84891,228039,832875,46603,2013-08-30 11:29:35,2013-08-30 11:30:05.517,algebra,1,...,30495,29881,NaN,-19,0,146118011,0.361323,0.0,0.211268,1.000000
6123268,146118739,NaN,543973,228055,832844,331778,2013-08-30 11:52:37,2013-08-30 11:52:41.251,choose_1,1,...,4244,331778,NaN,Yes,0,146118739,0.361323,0.0,0.336529,0.442968


Initialize Concept Subset from Nodes

In [ ]:
nodes = ['316.0', '86.0', '318.0', '82.0', '278.0', '276.0', '62.0', '54.0', '49.0', '58.0', '74.0', '95.0', '358.0', '104.0', '91.0', '48.0', '47.0', '280.0', '67.0', '64.0', '83.0', '84.0', '65.0', '50.0', '77.0', '25.0', '22.0', '61.0', '279.0', '309.0', '311.0', '11.0', '18.0', '17.0', '46.0', '15.0', '12.0', '13.0', '51.0', '24.0', '368.0', '16.0', '79.0', '81.0', '298.0', '76.0', '34.0', '312.0', '326.0', '310.0', '106.0', '277.0', '92.0', '322.0', '75.0', '78.0', '70.0', '323.0', '331.0', '27.0', '346.0', '366.0', '307.0', '4.0', '69.0', '315.0', '334.0', '317.0', '336.0', '301.0', '32.0', '53.0', '321.0', '338.0', '80.0', '166.0', '26.0', '296.0', '297.0', '295.0', '42.0', '303.0', '302.0', '5.0', '39.0', '90.0', '40.0', '85.0', '8.0', '371.0', '350.0', '325.0', '378.0', '96.0', '340.0', '306.0', '1.0', '283.0', '304.0', '294.0', '305.0', '288.0', '308.0', '314.0', '363.0', '356.0', '110.0', '94.0', '324.0', '105.0', '101.0', '333.0', '103.0', '299.0', '290.0', '7.0', '36.0', '320.0', '21.0', '35.0', '88.0', '233.0', '186.0', '98.0', '41.0', '213.0', '332.0', '14.0', '163.0', '281.0', '344.0', '343.0', '284.0', '523.0', '367.0', '198.0', '282.0', '327.0', '365.0', '293.0', '292.0', '571.0', '572.0', '569.0', '570.0', '573.0', '574.0', '568.0', '580.0', '582.0', '577.0', '597.0', '595.0', '596.0', '593.0', '594.0', '592.0', '389.0', '388.0', '583.0', '585.0', '584.0', '590.0', '589.0', '578.0', '576.0', '579.0', '581.0', '575.0', '588.0', '591.0', '586.0', '587.0', '319.0', '99.0', '10.0', '362.0', '375.0', '63.0', '203.0', '231.0', '240.0', '373.0', '355.0', '217.0', '9.0', '212.0', '204.0', '218.0', '196.0', '216.0', '97.0', '223.0', '181.0', '370.0', '174.0', '173.0', '206.0', '214.0', '2.0', '287.0', '392.0', '354.0', '6.0', '202.0', '238.0', '224.0', '192.0', '197.0', '33.0', '226.0', '393.0', '313.0', '165.0', '52.0', '391.0', '339.0', '376.0', '390.0', '1641.0', '93.0', '385.0', '300.0', '361.0', '387.0', '359.0', '222.0', '360.0', '184.0', '230.0', '345.0', '291.0', '603.0', '601.0', '602.0', '600.0', '605.0', '599.0', '598.0', '606.0', '608.0', '614.0', '613.0', '604.0', '607.0', '612.0', '609.0', '610.0', '289.0', '100.0', '348.0', '386.0', '3.0', '220.0', '177.0', '45.0', '43.0', '209.0', '38.0', '172.0', '23.0', '29.0', '611.0', '30.0', '178.0']
#nodes =
nodes = [float(node) for node in nodes]
len(nodes)

265

In [ ]:
import random
nodes = random.sample(nodes, 83)

In [ ]:
queried_df = assist_df[assist_df['skill_id'].isin(nodes)]
queried_df

,problem_log_id,skill,problem_id,user_id,assignment_id,assistment_id,start_time,end_time,problem_type,original,...,overlap_time,template_id,answer_id,answer_text,first_action,problemlogid,Average_confidence(FRUSTRATED),Average_confidence(CONFUSED),Average_confidence(CONCENTRATING),Average_confidence(BORED)
25,141491413,Square Root,401322,86324,678721,229084,2013-02-01 20:46:04,2013-02-01 20:46:08.747,algebra,1,...,4740,29962,NaN,6,0,141491413,0.361323,0.0,0.766925,0.800000
27,138932174,Finding y-intercept from Linear Equation,404999,88027,585918,232103,2012-10-31 11:43:43,2012-10-31 11:46:26.139,algebra,1,...,107599,61828,NaN,1,0,138932174,0.361323,0.0,0.459499,0.316406
60,144330014,Congruence,88948,92685,788546,50123,2013-05-03 12:56:53,2013-05-03 12:56:58.56,algebra,1,...,5555,31956,NaN,61,0,144330014,0.361323,0.0,0.766925,0.000000
80,145466433,Complementary and Supplementary Angles,88892,92709,821363,50067,2013-06-17 10:34:49,2013-06-17 10:35:01.06,algebra,1,...,12046,30029,NaN,65,0,145466433,0.361323,0.0,0.766925,0.762689
86,143142614,Solving Systems of Linear Equations by Graphing,128417,92710,711298,69027,2013-04-05 15:51:03,2013-04-05 15:51:19.856,algebra,1,...,16827,60010,NaN,-3,0,143142614,0.361323,0.0,0.766925,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6123200,145993637,NaN,12375,226273,822324,12375,2013-08-25 09:58:10,2013-08-25 09:58:43.435,choose_1,1,...,33420,12375,NaN,B. 1 to 12,0,145993637,0.361323,0.0,0.336529,0.442968
6123224,146028217,Multiplication and Division Positive Decimals,335499,226593,822340,183358,2013-08-26 22:46:42,2013-08-26 22:47:56.548,algebra,1,...,47999,98449,NaN,7,0,146028217,0.361323,0.0,0.489096,0.000000
6123240,146087512,Number Line,208920,227002,832044,107112,2013-08-29 10:33:58,2013-08-29 10:34:38.004,choose_1,1,...,39976,107112,NaN,A,0,146087512,0.361323,0.0,0.336529,0.442968
6123241,146058570,Pythagorean Theorem,140521,227039,831803,75462,2013-08-28 11:38:57,2013-08-28 11:41:01.95,algebra,1,...,124941,30006,NaN,5,0,146058570,0.361323,0.0,0.336529,0.442968


In [ ]:
queried_df.to_csv('/content/drive/MyDrive/Education_Data/ProblemBodies2023_2.0/2012-2013-data-with-predictions-4-final.csv')